In [104]:
using Random

## Generate Random Sequences

In [115]:
function Arti_DNA(t, l, k, m, gc)  
    DNA = []
    for i in 1:t
        rand_str = randstring("AT", 100-gc)*randstring("GC", gc)
        seq = randstring(rand_str, l)
        push!(DNA, seq)
    end
    #This might get lost in background. Should probably not make this random
    #motif = randstring("ACGT", k)
    motif = "AAAAAAAA"
    position = []
    for i in 1:t
        rand_num = rand([i for i = 1:l], m)
        r = sort(rand_num)
        p = []
        for n in 1:m
            DNA[i] = DNA[i][1:r[n]+k*(n-1)]*motif*DNA[i][r[n]+k*(n-1)+1:l+k*(n-1)]
            push!(p, r[n]+k*(n-1)+1)
        end
        push!(position, p)
    end
    return(position, motif, DNA)
end

Arti_DNA (generic function with 1 method)

In [125]:
t = 4
l = 20
k = 8
m = 1
gc = 41
(positions, motif, dna) = Arti_DNA(t, l, k, m, gc)
#dna = ["ATAAT","ATAAT","ATAAT"]

(Any[Any[17], Any[20], Any[6], Any[12]], "AAAAAAAA", Any["AGTGACATTCAGACATAAAAAAAAGCCT", "TTAACCCCAAAGGAGCATCAAAAAAAAC", "GCTTTAAAAAAAAAGGGAACCGACTTGT", "GCTATTTAAATAAAAAAAAACCAGCTTC"])

## Pair Motif

In [117]:
function d_hamm(x,x1)
    i=1
    count = 0
    while i<=length(x)
        if x[i]!=x1[i]
            count+=1
        end
        i+=1
    end
    return count
end

d_hamm (generic function with 1 method)

In [118]:
function get_C(x, dna, d)
    Cxsi = Dict()
    t = length(dna)
    l = length(dna[1])
    k = length(x)
    for i in 2:t
        C = Array[]
        for j in 1:(l-k+1)
            xp = dna[i][j:j+k-1]
            if d_hamm(x,xp) <= 2*d
               push!(C, [xp,j])
            end
        end
        Cxsi[i] = C
    end
    return Cxsi
end

get_C (generic function with 1 method)

In [119]:
function filter1(x, xp, z, d)
    d1 = 0
    d2 = 0
    for i in 1:length(z)
        if x[i] != z[i]
            d1 += 1
        end
        if xp[i] != z[i]
            d2 += 1
        end
    end
    if d1 > 2*d || d2 > 2*d
       return false 
    end
    return true
end

filter1 (generic function with 1 method)

In [120]:
function p_val(x,x1,z,d)
    i = 1
    count1 = 0
    count2 = 0
    while i<=length(x)
        if x[i]==x1[i]
            if z[i]!=x[i] || z[i]!=x1[i]
                count1+=1
            end
        else
            if z[i]!=x[i]  || z[i]!=x1[i]
                count2+=1
            end
        end
        i+=1
    end
    return [count1,count2]
end

p_val (generic function with 1 method)

In [121]:
function filter2(x,x1,z,d)
    alpha_range = length(x)-d_hamm(x,x1)
    beta_range = d_hamm(x,x1)

    p_values = p_val(x,x1,z,d)

    for i in 0:alpha_range
        for j in 0:beta_range
            k = 2*i+j+d_hamm(x,x1)
            if k<=2*d
                if abs(p_values[1]-i)+abs(p_values[2]-j)<=d
                    return true
                end
            end
        end
    end
    return false
end

filter2 (generic function with 1 method)

In [122]:
function get_all_md_kmers(x, xp, d, ch, seq, k, Md)
    if k == 0
        if d_hamm(seq, x) <= d && d_hamm(seq, xp) <= d
            push!(Md, seq)
        end
        return
    end
    
    for i in 1:length(ch)
        nseq = ""
        nseq = seq * ch[i]
        get_all_md_kmers(x, xp, d, ch, nseq, k - 1, Md)
    end
end

get_all_md_kmers (generic function with 1 method)

In [123]:
function verify(y, Cp, r, d)
    final_dict = Dict()
    for (i, C) in Cp
        valid = false
        if i != r
            for yi in C
                if d_hamm(y, yi[1]) <= d
                    final_dict[i] = [yi[1],yi[2]]
                    valid = true
                    break
                end
            end
            if !valid
                return [false,[]]
            end
        end
    end
    return [true,final_dict]
end

verify (generic function with 1 method)

In [132]:
d_thres = 1
s1 = dna[1]
M = []
motif_positions = Dict()
for i in 1:length(dna[1]) - k + 1
    x = dna[1][i:i+k-1]
    Cxsi = get_C(x, dna, d_thres)
    r = rand(2:t)
    sr = dna[r]
    Cxsr = Cxsi[r]
    Md = Dict()
    for xp in Cxsr
        pair = (x, xp[1])
        Md[pair] = []
        get_all_md_kmers(x, xp[1], d_thres, ["A","T","G","C"], "", k, Md[pair])
        
        Cpxsi = Dict()
        for zi in 2:length(dna)
            if zi != r
                Cpxsi[zi] = []
                for z in Cxsi[zi]
                    if filter1(x, xp[1], z[1], d_thres) && filter2(x, xp[1], z[1], d_thres)
                        push!(Cpxsi[zi], [z[1],z[2]])
                    end
                end
            end
        end
        for y in Md[pair]
            verification_results = verify(y, Cpxsi, r, d_thres)
            if verification_results[1] == true
                if !(y in M)
                    push!(M, y)
                    motif_positions[y] = verification_results[2]
                end
            end
        end
        
    end
end
println(M)
println(motif_positions)

Any["AAATAAAA", "AAAAAAAA", "ATAAAAAA", "ATTAAAAA", "AGAAAAAA", "ACAAAAAA", "TTAAAAAA", "CTAAAAAA", "TAAAAAAA", "TGAAAAAA", "TCAAAAAA", "TAAAAAAC", "GAAAAAAA", "CAAAAAAA", "AAAGAAAA", "AAACAAAA", "AATAAAAA", "AAGAAAAA", "AACAAAAA", "AAAAAAAT", "AAAAAAAG", "AAAAAAAC", "AAAAAATA", "AAAAAAGA", "AAAAAACA", "AAAAATAA", "AAAAAGAA", "AAAAACAA", "AAAATAAA", "AAAAGAAA", "AAAACAAA", "AAAAAAGC"]
Dict{Any,Any}("AAAAAACA" => Dict{Any,Any}(2 => Any["AAAAAAAA", 20],3 => Any["AAAAAAAA", 6]),"TGAAAAAA" => Dict{Any,Any}(4 => Any["TAAAAAAA", 11],2 => Any["TCAAAAAA", 18]),"AAAGAAAA" => Dict{Any,Any}(2 => Any["AAAAAAAA", 20],3 => Any["AAAAAAAA", 6]),"AAAAAAAA" => Dict{Any,Any}(2 => Any["CAAAAAAA", 19],3 => Any["TAAAAAAA", 5]),"AACAAAAA" => Dict{Any,Any}(2 => Any["ATCAAAAA", 17],3 => Any["AAAAAAAA", 6]),"TCAAAAAA" => Dict{Any,Any}(4 => Any["TAAAAAAA", 11],2 => Any["TCAAAAAA", 18]),"AAAAATAA" => Dict{Any,Any}(2 => Any["AAAAAAAA", 20],3 => Any["AAAAAAAA", 6]),"AAAACAAA" => Dict{Any,Any}(2 => Any["AAAAAAAA", 2